# Import libraries

In [ ]:
import time
import face_recognition
from matplotlib import pyplot as plt
import cv2
from glob import glob as gl
import numpy as np
import tensorflow as tf

from deepface import DeepFace
from deepface.commons import distance as dst
import time
import cv2
from glob import glob as gl
import numpy as np
from tqdm import trange

from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs, convert_output

num_classes = 1
tfrecord = None
weights = 'face-yolov3-tiny.tf'
classes = './yolov3_tf2/face.names'
size = 416
yolo = YoloV3Tiny(classes=num_classes)
yolo.load_weights(weights).expect_partial()

models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib"]
model_name = "Dlib"
model = DeepFace.build_model(model_name)

In [ ]:
def face_detection(img_raw):
    img_raw = tf.convert_to_tensor(img_raw, dtype=tf.uint8)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)
    boxes, scores, classes, nums = yolo(img)
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    output = convert_output(img, (boxes, scores, classes, nums))
    return output

In [ ]:
def distance(e1, e2):
    distance = dst.findEuclideanDistance(dst.l2_normalize(e1), dst.l2_normalize(e2))
    return distance
    


# Define some functions

In [ ]:
def load_user(path):
    images = gl(path+"/*.jpg")
    info_file = path+"/info.txt"
    f = open(info_file, "r")
    content = f.readlines()
    info = {"uid": content[0][:-1],
             "name": content[1][:-1],
             "age": content[2][:-1],
             "permission": content[3][:-1],
             "more": content[4][:-1]
            }
    encoded_faces = []
    for image in images:
        img = cv2.imread(image)
        encoded_face = DeepFace.represent(img, model = model, fast=True)
        encoded_faces.append(encoded_face)
        
    return {"info":info, "vectors": encoded_faces}
        
def load_data():
    paths = gl("users/*")
    data = {}
    for path in paths:
        user = load_user(path)
        data[user.get("info").get("uid")] = user
    return data

def distances_compute(face, user):
    h, w = face.shape[:2]
    encoded_face = DeepFace.represent(face, model = model, fast=True)
    vectors = user['vectors']
    distances = []
    for vector in vectors:
        distances.append(distance(vector, encoded_face))
    
    return np.array(distances)

def user_verify(face, threshold = 0.4):
    min_distance = 2
    found_key = None
    user_matched = None
    for key in keys:
        distances = distances_compute(face, data[key])
        mean_dis, min_dis = np.mean(distances), np.min(distances)
        if min_dis < min_distance:
            found_key = key
            min_distance = min_dis
            user_matched = data[found_key]["info"]
    if min_distance<threshold:
        return min_distance, user_matched
    else:
        return min_distance, None

# Load User data

In [ ]:
data = load_data()
keys = list(data.keys())

# Test with single image

In [ ]:
test_image = face_recognition.load_image_file("1.jpg")
import time
tik = time.time()
# faces detection
face_location = face_detection(test_image)[0]
tok = time.time()
print(tok -tik)
top, right, bottom, left = face_location
# crop face
face = test_image[top:bottom, left:right]
# face = cv2.imread("2.jpg")
plt.imshow(face)
plt.show()
tik = time.time()
# face verify 
min_distance, user_matched = user_verify(face, threshold = 1)
tok = time.time()
print(tok -tik)
print(min_distance)
print(user_matched)

# Test with video

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
# fontScale
fontScale = 0.7
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2
video_link = "1.mp4"
cap = cv2.VideoCapture(video_link)
stt = 0
t1 = time.time()
while True:
    t2 = time.time()
    fps = 1/(t2-t1)
    t1 = t2
    stt+=1
    ret, frame = cap.read()
#     frame = cv2.resize(frame, (700, 400))
    if not ret:
        break
    t3 = time.time()
    face_locations = face_detection(frame)
    t4 = time.time()
    N = len(face_locations)
    if N>0:
        for face_location in face_locations:
            top, right, bottom, left = face_location
            face = frame[top:bottom, left:right][:, :, ::-1]
            t5 = time.time()
            min_distance, user_matched = user_verify(face)
            t6 = time.time()
            user, rect_color = [user_matched['name'], (0, 255, 0)] if user_matched is not None else ["Unknown", (255, 0, 0)]
            if min_distance>0.6:
                rect_color = (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), rect_color, 2)
            cv2.putText(frame, user +"("+str(round(min_distance, 2))+")", (left, top -10), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS: "+str(round(fps)), (30, 30), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS2: "+str(round(t4-t3, 4)), (30, 60), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(frame, "FPS3: "+str(round(t6-t5, 4)), (30, 90), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow("frame", frame)
        k = cv2.waitKey(1)
        if k == ord('q'):
            cv2.destroyAllWindows()
            break

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import tensorflow as tf
print(tf.__version__)